In [ ]:
import geemap
import ee
ee.Initialize()
from datetime import datetime
import scripts.analysis_functions as af
import scripts.fire_info as fi
import scripts.get_image_collections as gic


In [ ]:
py_date = datetime.utcnow()
current_date = py_date.strftime("%Y-%m-%d")
fire_dict = fi.fire_dict
# dixie,tamarack,beckwourth
fire = "dixie"

region_str, fire_start, fire_end, pre_start, pre_end = fi.get_fire_by_name(fire, current_date)
region = ee.FeatureCollection(region_str)

print(fire_start,fire_end)

In [ ]:
# landsatCol : None, t2, bestls
# None=T1 real time toa, t2=None+ T1 real time toa, bestls=T1 toa
# cloudBustingMethod : None, bust
# None=No cloud masking/busting, bust=cloud busting
# sensor : landsat, sentinel2
sensor = "landsat"
pre_collection = gic.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
print(pre_collection.size().getInfo())
pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)
print(pre_img.bandNames().getInfo())

post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol=None)
print(post_collection.size().getInfo())
post_img = gic.get_composite(post_collection,gic.make_nrt_composite,sensor)
print(post_img.bandNames().getInfo())

In [ ]:
Map = geemap.Map()
Map

In [ ]:
Map.addLayer(pre_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"pre")
Map.addLayer(post_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"post")
Map.addLayer(region)

In [ ]:
rdnbr = af.rdnbr(pre_img,post_img)
miller = af.miller_thresholds(rdnbr)
Map.addLayer(rdnbr.clip(region),{"min":0,"max":1500,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'rdnbr')
Map.addLayer(miller.clip(region),{"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'miller')

In [ ]:
import scripts.utils as utils

desc = utils.default_export_name(fire, fire_end, fire_start, pre_end, pre_start) + "_cog"
prefix = f'fire_response_2021/{sensor}'
print(desc)
img = miller.addBands(rdnbr).addBands(post_img.select("date")).clip(region)
utils.exportMapToCloud(img.toDouble(),desc,region.geometry(),prefix=prefix,default=sensor)


Optionally, it is possible to use the analysis functions to create NBR or dNBR insead of the RdNBR 

In [ ]:
# getting dNBR by instead of RdNBR
pre_nbr = af.nbr(pre_img)
ntr_nbr = af.nbr(post_img)
dnbr = af.dnbr(pre_nbr,ntr_nbr)
Map.addLayer(dnbr,{"min":-1000,"max":1000,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'dnbr')

In [ ]:
import scripts.utils as utils

desc = utils.default_export_name(fire, fire_end, fire_start, pre_end, pre_start) + "_dnbr"
prefix = f'fire_response_2021/{sensor}'
print(desc)
img = dnbr.addBands(post_img.select("date")).clip(region)
utils.exportMapToCloud(img.toDouble(),desc,region.geometry(),prefix=prefix,default=sensor)
